# IMDB 영화 리뷰 감성 분석 - LSTM

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
import warnings
warnings.filterwarnings('ignore')

### Conv1D로 IMDB 리뷰 감성 분석
- 단어 빈도수 : 10,000 (총 88,584)
- 문장의 단어수 : 500 (최대 2,494)
- test data 중 40%(10,000개)는 검증용으로

In [2]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
X_train.shape, X_test.shape, y_train.shape

17473536/17464789 [==============================] - 0s 0us/step


((25000,), (25000,), (25000,))

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 500), (25000, 500))

In [6]:
from sklearn.model_selection import train_test_split
X_test, X_valid, y_test, y_valid = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.4, random_state=seed
)
X_test.shape, X_valid.shape, y_test.shape, y_valid.shape

((15000, 500), (10000, 500), (15000,), (10000,))

- Case 1) Conv1D x 2, MaxPooling1D x 2, Dropout, GlobalMaxPooling1D
    - embedding dim : 100

In [7]:
model1 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [8]:
model1.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model_path = 'best-imdb-conv1d.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [9]:
hist1 = model1.fit(
    X_train, y_train, epochs=30, batch_size=64, 
    validation_data=[X_valid, y_valid], 
    callbacks=[es, mc]
)

Epoch 1/30
391/391 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4999
Epoch 00001: val_loss improved from inf to 0.00000, saving model to best-imdb-conv1d.h5
391/391 [==============================] - 23s 23ms/step - loss: 0.0000e+00 - accuracy: 0.4999 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/30
388/391 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4995
Epoch 00002: val_loss did not improve from 0.00000
391/391 [==============================] - 9s 22ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/30
391/391 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 00003: val_loss did not improve from 0.00000
391/391 [==============================] - 8s 21ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/30
391/391 [==============================] - ETA: 0s - loss: 0.0000e+00 - accura

In [10]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 3s 6ms/step - loss: 0.0000e+00 - accuracy: 0.5000


[0.0, 0.5]

- Case 2) Conv1D + LSTM

In [11]:
from tensorflow.keras.layers import LSTM

In [12]:
model2 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

In [13]:
model2.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model_path = 'best-imdb-conv1d.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [14]:
hist2 = model2.fit(
    X_train, y_train, epochs=30, batch_size=64, 
    validation_data=[X_valid, y_valid], 
    callbacks=[es, mc]
)

Epoch 1/30
391/391 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 00001: val_loss improved from inf to 0.00000, saving model to best-imdb-conv1d.h5
391/391 [==============================] - 19s 42ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/30
390/391 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 00002: val_loss did not improve from 0.00000
391/391 [==============================] - 17s 44ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/30
391/391 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 00003: val_loss did not improve from 0.00000
391/391 [==============================] - 16s 42ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/30
391/391 [==============================] - ETA: 0s - loss: 0.0000e+00 - accu

In [15]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 7s 14ms/step - loss: 0.0000e+00 - accuracy: 0.5000


[0.0, 0.5]

- Case 3) Conv1D + Dense

In [16]:
model3 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)              

In [17]:
model3.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model_path = 'best-imdb-conv1d.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [18]:
hist3 = model3.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid], 
    callbacks=[es, mc]
)

Epoch 1/30
194/196 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4992
Epoch 00001: val_loss improved from inf to 0.00000, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 7s 28ms/step - loss: 0.0000e+00 - accuracy: 0.4997 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/30
196/196 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 00002: val_loss did not improve from 0.00000
196/196 [==============================] - 5s 25ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/30
195/196 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5001
Epoch 00003: val_loss did not improve from 0.00000
196/196 [==============================] - 5s 25ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/30
196/196 [==============================] - ETA: 0s - loss: 0.0000e+00 - accurac

In [19]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 3s 5ms/step - loss: 0.0000e+00 - accuracy: 0.5000


[0.0, 0.5]